# Dataset Merge IDEA

1. history에 대해 drop duplicate 후 drop column      
2. watch 에 대해서도 drop duplicate 후 drop column     
3. 이후 inner join       
4. 다른 dataset merge   

문제 : 3번부터 램이 터져나감    
대안 1. 나눠서 조인 (노가다...)   
대안 2. 더 나은 노트북 환경 찾기   
대안 3. 똑똑하게 머리 쓰기 **<해결 방안 찾아냄>**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

### 시청 시작 데이터에서 row 중복 체크 - 1만 6천개 가량의 중복 확인. Drop

In [2]:
'''
의심의 여지 없이 중복 존재 
아래 코드 실행 시 groupby로 상당히 많이 묶이고, 최종 column 개수가 899021로 줄었음을 알 수 있음
''' 
# history.groupby(['profile_id','log_time','ss_id']).count().sort_values('album_id',ascending=False)[:60]

'\n의심의 여지 없이 중복 존재 \n아래 코드 실행 시 groupby로 상당히 많이 묶이고, 최종 column 개수가 899021로 줄었음을 알 수 있음\n'

In [3]:
history = pd.read_csv('data/history_data.csv')
history.drop_duplicates(subset=['profile_id','log_time','album_id'],inplace=True) 
# row = 1005651 rows × 8 columns
# after drop duplicates 899021 rows × 8 columns
# 106630 개 row 중복

In [4]:
# 'continuous_play','short_trailer','log_time' -- drop
# payment 결측 0으로 대체
# history.drop(labels=['log_time','act_target_dtl','continuous_play','short_trailer'],axis=1).fillna(0,inplace=True)

### Watch에 대해서도 동일한 작업 수행 (duplicate drop, feature selection)

In [5]:
watch = pd.read_csv('data/watch_e_data.csv')
watch.drop_duplicates(subset=['profile_id','log_time','album_id'],inplace=True) 
# watch.drop(labels=['log_time','act_target_dtl'],axis=1,inplace=True)  # column 드랍. 따로 처리할 결측치 없음

In [6]:
# full 병합 코드는 폭탄 그 자체
# 돌렸다 하면 터져용 
# datamerge = pd.merge(history,watch,how='inner',on='profile_id')
# datamerge.to_csv('../input/lgground')

## 터진다...! 💣💥
*근데 여기서 문득 드는 생각*

1. 반드시 history와 watch를 inner join으로 묶어야 할까? 그러니까... serve할 비중복 유저 리스트만 history에서 가져오고, 제대로 된 neg/pos sampling이 가능한 watch 데이터에 붙여서 그것 중심으로 학습할 수도 있지 않나?     
애초에 history에서 유의미한 column은 **시청 시작 데이터에만 있는 profile_id & 그나마 payment 뿐이다**    
ㄴ *그나마 payment* : payment는 영상 시청의 장벽 같은 요소로 작동하기 때문에 (무료 영상이 유료 영상보다 많이 재생된다) sampling 혹은 추천 요소에 중요하게 고려할 만 하다.    
.       
           
이렇게 접근하면 어떤 방식이 가능해지냐면 < log_time column도 날릴 수 있게 된다 :: 그래도 train-vaild를 위해서 날리진 말자 >    
1. 두 dataset에서 필요 없는 column을 drop하고       
2. history에만 있는 profile_id 가 있는 row를 전부 추출해 watch에 append한다.
3. 추후 데이터셋 분할을 고려해, log_time 기준으로 dataframe 정렬 sort.    
3. 위 데이터셋에 meta + profile 데이터셋 merge 수행한다.    
     
> 최종적으로는 watch에 있는 중요한 값을 기본으로 가지고, serve대상인 유저목록까지 챙긴 dataset이 됨     

In [7]:
# data preprocessing
# history에만 있는 profile_id  Length: 8311 / watch에만 있는 profile_id  Length: 7658
m_his = history['profile_id'].drop_duplicates() # 터짐 방지로 profile_id column만 남김 
m_wat = watch['profile_id'].drop_duplicates()
id_only_in_history = pd.merge(m_his,m_wat,how='outer',indicator=True
                ).query('_merge == "left_only"').drop(columns=['_merge'])
id_only_in_history_list = id_only_in_history['profile_id'].to_list()
id_only_in_history_rows = history[history['profile_id'].isin(id_only_in_history_list)]
id_only_in_history_rows = id_only_in_history_rows.drop_duplicates(subset=['profile_id','ss_id','log_time','album_id']) # 15241 rows
id_only_in_history_rows.drop(columns=['continuous_play','short_trailer'])
dataset = watch.append(id_only_in_history_rows,sort=False).sort_values('log_time').reset_index(drop=True)

/var/folders/wz/2dv1p69j3pv2c66bj9jt_hbc0000gn/T/ipykernel_61340/2297796919.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = watch.append(id_only_in_history_rows,sort=False).sort_values('log_time').reset_index(drop=True)


In [8]:
# dataset verification
# watch : 800632 +15241 = row 815873 나와야 정상병합
print(dataset['total_time'].isnull().sum()) #제대로 합쳐졌다면 결측이 15241개 있을 것
print(dataset['profile_id'].nunique()) #제대로 합쳐졌다면 8311명일 것 

15241
8311


In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815873 entries, 0 to 815872
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   profile_id       815873 non-null  int64  
 1   ss_id            815873 non-null  int64  
 2   log_time         815873 non-null  int64  
 3   act_target_dtl   815873 non-null  object 
 4   album_id         815873 non-null  int64  
 5   watch_time       800632 non-null  float64
 6   total_time       800632 non-null  float64
 7   continuous_play  815873 non-null  object 
 8   payment          564 non-null     float64
 9   short_trailer    15241 non-null   object 
dtypes: float64(3), int64(4), object(3)
memory usage: 62.2+ MB


# Meta Data merge 

In [10]:
# # 메타데이터 병합 
meta = pd.read_csv('data/meta_data.csv') # 42602 rows
meta_p = pd.read_csv('data/meta_data_plus.csv') # 767948 rows
# meta_merge = pd.merge(meta,meta_p,how='inner',on='album_id') #### meta 정보 최종 집합체 : 832356 rows
# data_w_meta = pd.merge(dataset,meta,how='inner',on='album_id')

/var/folders/wz/2dv1p69j3pv2c66bj9jt_hbc0000gn/T/ipykernel_61340/3753341068.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv('data/meta_data.csv') # 42602 rows


# make data set

In [11]:
# 하이퍼파라미터 
class cfg: 
    gpu_idx = 0
    top_k = 25
    seed = 42
    neg_ratio = 100
    test_size = 0.2

In [12]:
data = dataset[['profile_id','album_id']]
data['rating'] = 1

/var/folders/wz/2dv1p69j3pv2c66bj9jt_hbc0000gn/T/ipykernel_61340/2801991363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rating'] = 1


In [13]:
cfg.n_users = data.profile_id.max()+1
cfg.n_items = data.album_id.max()+1

from sklearn.model_selection import train_test_split
from scipy import sparse
from tqdm.notebook import tqdm
# 학습 및 검증 데이터 분리
train, valid = train_test_split(
    data, test_size=cfg.test_size, random_state=cfg.seed,
)
print('학습 데이터 크기:', train.shape)
print('검증 데이터 크기:', valid.shape)

학습 데이터 크기: (652698, 3)
검증 데이터 크기: (163175, 3)


In [14]:
# Matrix 형태로 변환 
train = train.to_numpy()
matrix = sparse.lil_matrix((cfg.n_users, cfg.n_items))
for (p, i, r) in tqdm(train):
    matrix[p, i] = r
    
train = sparse.csr_matrix(matrix)
train = train.toarray()
print("train 형태: \n", train)

  0%|          | 0/652698 [00:00<?, ?it/s]

train 형태: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [15]:
def make_UIdataset(train, neg_ratio):
    """ 유저별 학습에 필요한 딕셔너리 데이터 생성 
    Args:
        train : 유저-아이템의 상호작용을 담은 행렬 
            ex) 
                array([[0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        ...,
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.]])
        neg_ratio : negative sampling 활용할 비율 
            ex) 3 (positive label 1개당 negative label 3개)
    Returns: 
        UIdataset : 유저별 학습에 필요한 정보를 담은 딕셔너리 
            ex) {'사용자 ID': [[positive 샘플, negative 샘플], ... , [1, 1, 1, ..., 0, 0]]}
                >>> UIdataset[3]
                    [array([   16,    17,    18, ...,  9586, 18991,  9442]),
                    array([5, 5, 5, ..., 5, 5, 5]),
                    array([4, 4, 4, ..., 5, 1, 1]),
                    array([1., 1., 1., ..., 0., 0., 0.])]
    """
    UIdataset = {'profile_id': [], 'album_id':[],'rating':[]}
    for user_id, items_by_user in enumerate(train):
#         UIdataset[user_id] = []
        # positive 샘플 계산 
        pos_item_ids = np.where(items_by_user > 0.5)[0]
        num_pos_samples = len(pos_item_ids)

        # negative 샘플 계산 (random negative sampling) 
        num_neg_samples = neg_ratio * num_pos_samples

        neg_items = np.where(items_by_user < 0.5)[0]
        neg_item_ids = np.random.choice(neg_items, min(num_neg_samples, len(neg_items)), replace=False)
        UIdataset['album_id'] += list(neg_item_ids)+list(pos_item_ids)
        UIdataset['profile_id'] += [user_id for _ in range(len(neg_item_ids)+len(pos_item_ids))]

#         for column in columns:
#             # feature 추출 
#             features = []
#             for item_id in np.concatenate([pos_item_ids, neg_item_ids]): 
#                 features.append(user_features[column][user_id])
#             UIdataset[column].append(np.array(features))

#             features = []
#             for item_id in np.concatenate([pos_item_ids, neg_item_ids]): 
#                 features.append(item_features[column][item_id])
#             UIdataset[column].append(np.array(features))

            # label 저장  
        pos_labels = np.ones(len(pos_item_ids))
        neg_labels = np.zeros(len(neg_item_ids))
        UIdataset['rating'] += list(neg_labels)+list(pos_labels)

    return UIdataset

In [16]:
pos_neg_samplings = make_UIdataset(train,100)
pos_neg_samplings_df = pd.DataFrame(pos_neg_samplings)

In [17]:
meta_df = meta[['album_id','title','genre_mid','run_time']].drop_duplicates(subset=['album_id'], ignore_index=True)
profile = pd.read_csv('data/profile_data.csv')
profile_df = profile[['profile_id','sex','age']]

dataset_meta_merge = pd.merge(pos_neg_samplings_df, meta_df, how='left', on='album_id')
train_df = pd.merge(dataset_meta_merge,profile_df, how='left', on='profile_id')
qids_train = train_df.groupby('profile_id')['profile_id'].count().to_numpy()

In [18]:
valid_dataset_meta_merge = pd.merge(valid, meta_df, how='left', on='album_id')
valid_df = pd.merge(valid_dataset_meta_merge,profile_df, how='left', on='profile_id')
qids_validation = valid_df.groupby('profile_id')['profile_id'].count().to_numpy()

In [ ]:
train_df = pd.get_dummies(train_df, columns=['album_id','title','genre_mid','sex'],sparse=True)
valid_df = pd.get_dummies(valid_df, columns=['album_id','title','genre_mid','sex'],sparse=True)

# lightgbm rank

In [ ]:
import numpy as np
import pandas as pd
import lightgbm


X_train = train_df.drop(columns=['rating'])
y_train = train_df['rating']

X_validation = valid_df.drop(columns=['rating'])
y_validation = valid_df['rating']

In [ ]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
)

In [ ]:
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_validation, y_validation)],
    eval_group=[qids_validation],
    eval_at=10,
    verbose=10,
)